In [3]:
from pyspark.sql import DataFrame, Row
from functools import reduce
from pyspark.sql.types import *
from pyspark.sql.functions import *
from graphframes import *
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
import uuid

In [4]:
conf=SparkConf()
# conf.set("spark.executor.memory", "2g")
# conf.set("spark.driver.memory", "12g")
# conf.set("spark.cores.max", "6")

sc = SparkContext.getOrCreate(conf)

spark = SQLContext(sc)

23/12/11 09:44:35 WARN Utils: Your hostname, preacher resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface enp4s0)
23/12/11 09:44:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/11 09:44:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/preacher/anaconda3/envs/deltarun/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
    def prune_1p_3p_match_suggestions(self, match_suggestion: DataFrame):
        filtered_match_suggestion = None
        pruned_match_suggestion = None
        if self.config['seller_type'] == '1p':
            filtered_match_suggestion = match_suggestion.filter(col('seller_type') == '1p')
            pruned_match_suggestion = match_suggestion.filter(col('seller_type') == '3p')
            pruned_match_suggestion = pruned_match_suggestion.withColumn("misc_info", lit("pruned_by_seller_type_1p"))
        elif self.config['seller_type'] == '3p':
            filtered_match_suggestion = match_suggestion.filter(col('seller_type') == '3p')
            pruned_match_suggestion = match_suggestion.filter(col('seller_type') == '1p')
            pruned_match_suggestion = pruned_match_suggestion.withColumn("misc_info", lit("pruned_by_seller_type_3p"))
        else:
            filtered_match_suggestion = match_suggestion
        return filtered_match_suggestion, pruned_match_suggestion
    
    def separate_audited_unaudited_base_product_matches(self, match_suggestion:DataFrame, mdw:DataFrame):
        mdw = mdw.select("base_sku_uuid")
        unaudited_base_product_matches = match_suggestion.join(mdw, 'base_sku_uuid', 'left_anti')
        audited_base_product_matches = match_suggestion.join(mdw, 'base_sku_uuid', 'inner')
        return unaudited_base_product_matches, audited_base_product_matches

In [5]:
matches_schema = StructType([
    StructField("pair_id", StringType(), True),
    StructField("base_sku_uuid", StringType(), True),
    StructField("base_source_store", StringType(), True),
    StructField("comp_sku_uuid", StringType(), True),
    StructField("comp_source_store", StringType(), True),
    StructField("seller_type", StringType(), True),
])

mdw_schema = StructType([
    StructField("pair_id", StringType(), True),
    StructField("base_sku_uuid", StringType(), True),
    StructField("base_source_store", StringType(), True),
    StructField("comp_sku_uuid", StringType(), True),
    StructField("comp_source_store", StringType(), True),
    StructField("seller_type", StringType(), True),
    StructField("match_type", StringType(), True)
])

In [6]:
matche_data = [
    ["new1<>203359001<>homedepot<>homedepot_compNoSugg1p1<>g4103325<>bcg<>bcg","new1<>203359001<>homedepot<>homedepot","homedepot<>homedepot","compNoSugg1p1<>g4103325<>bcg<>bcg", "bcg<>bcg","1P"],
    ["new2<>100341666<>homedepot<>homedepot_compNoSugg1p2<>109589<>bcg<>bcg","new2<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compNoSugg1p2<>109589<>bcg<>bcg", "bcg<>bcg","1P"],
    
    [ "1<>203370169<>homedepot<>homedepot_compYes1pExactSugg1p<>967446<>bcg<>bcg","1<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compYes1pExactSugg1p<>967446<>bcg<>bcg","bcg<>bcg","1P"],
    [ "2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg","2<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compYes1pSimSugg1p<>967493<>bcg<>bcg","bcg<>bcg","1P"],
    [ "3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg","3<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compYes3pSimSugg3p<>109589<>bcg<>bcg","bcg<>bcg","3P"],
    [ "4<>53409<>petcarerx<>petcarerx_compYes3pExactSugg3p<>147664<>chewy<>chewy","4<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","compYes3pExactSugg3p<>147664<>chewy<>chewy", "chewy<>chewy","3P"],
    
    ["new3<>53409<>petcarerx<>petcarerx_compNoSugg3p1<>147664<>chewy<>chewy","new3<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","compNoSugg3p1<>147664<>chewy<>chewy", "chewy<>chewy","3P"],
    ["new4<>100341666<>homedepot<>homedepot_compNoSugg3p2<>109589<>bcg<>bcg","new4<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compNoSugg3p2<>109589<>bcg<>bcg", "bcg<>bcg","3P"],

    ["1<>203370169<>homedepot<>homedepot_compNo1pExactSugg1p<>967446<>bcg<>bcg","1<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pExactSugg1p<>967446<>bcg<>bcg","bcg<>bcg","1P"],
    ["2<>100670057<>homedepot<>homedepot_compNo1pSimSugg1p<>967493<>bcg<>bcg","2<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pSimSugg1p<>967493<>bcg<>bcg","bcg<>bcg","1P"],
    ["3<>100341666<>homedepot<>homedepot_compNo3pSimSugg3p<>109589<>bcg<>bcg","3<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compNo3pSimSugg3p<>109589<>bcg<>bcg","bcg<>bcg","3P"],
    ["4<>53409<>petcarerx<>petcarerx_compNo3pExactSugg3p<>147664<>chewy<>chewy","4<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","compNo3pExactSugg3p<>147664<>chewy<>chewy", "chewy<>chewy","3P"],

    [ "1<>203370169<>homedepot<>homedepot_compNo1pExactSugg3p<>967446<>bcg<>bcg","1<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pExactSugg3p<>967446<>bcg<>bcg","bcg<>bcg","3P"],
    [ "2<>100670057<>homedepot<>homedepot_compNo1pSimSugg3p<>967493<>bcg<>bcg","2<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pSimSugg3p<>967493<>bcg<>bcg","bcg<>bcg","3P"],
    [ "3<>100341666<>homedepot<>homedepot_compNo3pSimSugg1p<>109589<>bcg<>bcg","3<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compNo3pSimSugg1p<>109589<>bcg<>bcg","bcg<>bcg","1P"],
    [ "4<>53409<>petcarerx<>petcarerx_compNo3pExactSugg1p<>147664<>chewy<>chewy","4<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","compNo3pExactSugg1p<>147664<>chewy<>chewy", "chewy<>chewy","1P"],
    
    [ "5<>203370169<>homedepot<>homedepot_compNo1pExactSugg1p<>967446<>bcg<>bcg","5<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pExactSugg1p<>967446<>bcg<>bcg","bcg<>bcg","1P"],
    [ "5<>203370169<>homedepot<>homedepot_compNo3pExactSugg3p<>147664<>chewy<>chewy","5<>203370169<>homedepot<>homedepot","homedepot<>homedepot","compNo3pExactSugg3p<>147664<>chewy<>chewy", "chewy<>chewy","3P"],
    [ "5<>203370169<>homedepot<>homedepot_compNo1pExactSugg3p<>967446<>bcg<>bcg","5<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pExactSugg3p<>967446<>bcg<>bcg","bcg<>bcg","3P"],
    [ "5<>203370169<>homedepot<>homedepot_compNo3pExactSugg1p<>147664<>chewy<>chewy","5<>203370169<>homedepot<>homedepot","homedepot<>homedepot","compNo3pExactSugg1p<>147664<>chewy<>chewy", "chewy<>chewy","1P"],
    
    [ "6<>100670057<>homedepot<>homedepot_compNo1pSimSugg1p<>967493<>bcg<>bcg","6<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pSimSugg1p<>967493<>bcg<>bcg","bcg<>bcg","1P"],
    [ "6<>100670057<>homedepot<>homedepot_compNo3pSimSugg3p<>967493<>bcg<>bcg","6<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compNo3pSimSugg3p<>967493<>bcg<>bcg","bcg<>bcg","3P"],
    [ "6<>100670057<>homedepot<>homedepot_compNo1pSimSugg3p<>967493<>bcg<>bcg","6<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pSimSugg3p<>967493<>bcg<>bcg","bcg<>bcg","3P"],
    [ "6<>100670057<>homedepot<>homedepot_compNo3pSimSugg1p<>967493<>bcg<>bcg","6<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compNo3pSimSugg1p<>967493<>bcg<>bcg","bcg<>bcg","1P"],
]
matches = spark.createDataFrame(matche_data, schema=matches_schema)

mdw_data = [
    [ "1<>203370169<>homedepot<>homedepot_compYes1pExactSugg1p<>967446<>bcg<>bcg","1<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compYes1pExactSugg1p<>967446<>bcg<>bcg","bcg<>bcg","1P","exact"],
    [ "2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg","2<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compYes1pSimSugg1p<>967493<>bcg<>bcg","bcg<>bcg","1P","similar"],
    [ "3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg","3<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compYes3pSimSugg3p<>109589<>bcg<>bcg","bcg<>bcg","3P","similar"],
    [ "4<>53409<>petcarerx<>petcarerx_compYes3pExactSugg3p<>147664<>chewy<>chewy","4<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","compYes3pExactSugg3p<>147664<>chewy<>chewy", "chewy<>chewy","3P","exact"],
    
    [ "5<>203370169<>homedepot<>homedepot_compYes1p3pExactSugg1p<>967446<>bcg<>bcg","5<>203370169<>homedepot<>homedepot","homedepot<>homedepot","compYes1p3pExactSugg1p<>967446<>bcg<>bcg","bcg<>bcg","1P","exact"],
    [ "5<>203370169<>homedepot<>homedepot_compYes3p3pExactSugg3p<>147664<>bcg<>bcg","5<>203370169<>homedepot<>homedepot","homedepot<>homedepot","compYes3p3pExactSugg3p<>147664<>bcg<>bcg", "bcg<>bcg","3P","exact"],

    [ "6<>100670057<>homedepot<>homedepot_compYes1p3pSimSugg1p<>967493<>bcg<>bcg","6<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compYes1p3pSimSugg1p<>967493<>bcg<>bcg","bcg<>bcg","1P","similar"],
    [ "6<>100670057<>homedepot<>homedepot_compYes1p3pSimSugg3p<>967493<>bcg<>bcg","6<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compYes1p3pSimSugg3p<>967493<>bcg<>bcg","bcg<>bcg","3P","similar"],
]
mdw = spark.createDataFrame(mdw_data, schema=mdw_schema)

In [7]:
matches.show()
mdw.show(truncate= False)

+--------------------+--------------------+--------------------+--------------------+-----------------+-----------+
|             pair_id|       base_sku_uuid|   base_source_store|       comp_sku_uuid|comp_source_store|seller_type|
+--------------------+--------------------+--------------------+--------------------+-----------------+-----------+
|new1<>203359001<>...|new1<>203359001<>...|homedepot<>homedepot|compNoSugg1p1<>g4...|         bcg<>bcg|         1P|
|new2<>100341666<>...|new2<>100341666<>...|homedepot<>homedepot|compNoSugg1p2<>10...|         bcg<>bcg|         1P|
|1<>203370169<>hom...|1<>203370169<>hom...|homedepot<>homedepot|compYes1pExactSug...|         bcg<>bcg|         1P|
|2<>100670057<>hom...|2<>100670057<>hom...|homedepot<>homedepot|compYes1pSimSugg1...|         bcg<>bcg|         1P|
|3<>100341666<>hom...|3<>100341666<>hom...|homedepot<>homedepot|compYes3pSimSugg3...|         bcg<>bcg|         3P|
|4<>53409<>petcare...|4<>53409<>petcare...|petcarerx<>petcarerx|compYes3

In [8]:
def combine_dfs(df_list: list[DataFrame]) -> DataFrame:
    print("Combining dfs")
    if df_list is None:
        return None
    
    df_list = [df for df in df_list if df is not None]
    if len(df_list) == 0:
        return None
    if len(df_list) == 1:
        return df_list[0]
    combined_df = reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), df_list)
    return combined_df

In [35]:
from pyspark.sql.window import Window

class Match_Pruner:
    def __init__(self, mdw: DataFrame, match_suggestion: DataFrame, config: dict) -> None:
        self.mdw = mdw
        self.match_suggestion = match_suggestion
        self.config = config

    def prune_matches(self):
        self.mdw, match_suggestion = self.preprocessing_matches(self.mdw, self.match_suggestion)
        # filtered_match_suggestion, _1p_3p_pruned_match_suggestion = self.prune_1p_3p_match_suggestions(match_suggestion)
        # print("filtered_match_suggestion")
        # filtered_match_suggestion.show()
        # if _1p_3p_pruned_match_suggestion is not None:
        #     print("_1p_3p_pruned_match_suggestion")
        #     _1p_3p_pruned_match_suggestion.show()
        # unaudited_base_match_suggestion, audited_base_product_matches = self.separate_audited_unaudited_base_product_matches(filtered_match_suggestion, self.mdw)
        # unaudited_base_match_suggestion, audited_base_product_matches = self.separate_audited_unaudited_base_product_matches(match_suggestion, self.mdw)
        # print("unaudited_base_match_suggestion")
        # unaudited_base_match_suggestion.show()
        # print("audited_base_product_matches")
        # audited_base_product_matches.show()

        similar_matches, exact_matches, unaudited_matches = self.separate_audited_and_unaudited_matches(match_suggestion, self.mdw)
        print("similar_matches")
        similar_matches.show(truncate=False)
        print("exact_matches")
        exact_matches.show(truncate=False)
        print("unaudited_matches")
        unaudited_matches.show(truncate=False)

        pruned_suggestion, match_suggestion = self.prune_match_based_on_match_type(similar_matches, exact_matches, unaudited_matches)
            
        if combine_dfs([pruned_suggestion, match_suggestion]).count() == unaudited_matches.count():
            print("matches are not missing")
        elif combine_dfs([pruned_suggestion, match_suggestion]).count() < unaudited_matches.count(): 
            print("matches are missing")
        else :
            print("duplicate matches are present")

        # pruned_suggestion = combine_dfs([_1p_3p_pruned_match_suggestion, match_type_pruned_suggestion])
        # match_suggestion = combine_dfs([unaudited_base_match_suggestion, unaudited_match_type_suggestion])

        return match_suggestion, pruned_suggestion

    def preprocessing_matches(self, mdw: DataFrame, match_suggestion: DataFrame):
        match_suggestion = match_suggestion.withColumn("seller_type", lower(col("seller_type")))
        mdw = mdw.withColumn("seller_type", lower(col("seller_type")))\
            .withColumn("bungee_audit_status", when( col("bungee_audit_status").isNotNull(), lower(col("bungee_audit_status")) ).otherwise(col("bungee_audit_status")) )

        return mdw, match_suggestion
    
    def separate_audited_and_unaudited_matches(self, match_suggestion:DataFrame, mdw:DataFrame):
        print("separate_audited_and_unaudited_matches")
        # audited_base_product_matches.show(truncate=False)
        # audited_matches = audited_base_product_matches.select("pair_id").join(mdw, 'pair_id', 'inner')
        # audited_matches.show(truncate=False)
        window_spec = Window.partitionBy("base_sku_uuid")

        similar_matches = mdw.filter(col('bungee_audit_status').isin(['equivalent', 'similar']))
        similar_matches = similar_matches.withColumn("audited_match_seller_type_list", concat_ws( "_", array_sort( collect_set(col("seller_type")).over(window_spec) ) ) )
        similar_matches = similar_matches.withColumn("misc_info", concat(lit("similar_match_"), col("pair_id")) ).\
            select("base_sku_uuid", "audited_match_seller_type_list", "misc_info", col("seller_type").alias("audited_match_seller_type"))
        similar_matches.show(n=10)                                
               
        exact_matches = mdw.filter(col('bungee_audit_status').isin(['exact']))
        exact_matches = exact_matches.withColumn("audited_match_seller_type_list", concat_ws("_", array_sort( collect_set(col("seller_type")).over(window_spec) ) ) )
        exact_matches = exact_matches.withColumn("misc_info", concat(lit("exact_match_"), col("pair_id")) ).\
                        select("base_sku_uuid", "audited_match_seller_type_list","misc_info", col("seller_type").alias("audited_match_seller_type"))
        exact_matches.show(n=10) 
        unaudited_matches = match_suggestion.join(mdw.select("pair_id"), 'pair_id', 'left_anti')
        return similar_matches, exact_matches, unaudited_matches

    def prune_match_based_on_match_type(self, similar_matches:DataFrame , exact_matches:DataFrame , audited_base_product_unaudited_matches:DataFrame ):
        # audited_base_product_unaudited_matches contains only unaudited matches 
        pruned_suggestion = None
        match_suggestion = None
        if self.config['match_type'] == 'exact' or self.config['match_type'] == 'exact_over_similar':
            if self.config['seller_type'] == '1p':
                audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_matches.filter(col("audited_match_seller_type") == '1p'), "base_sku_uuid", "left")
                pruned_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNotNull() | (col("seller_type") == '3p')).\
                                    withColumn("misc_info", when( col("seller_type") == '3p', lit("seller_type_config_1p")).otherwise(col("misc_info")) )
                match_suggestion = audited_base_product_unaudited_matches.filter( col("misc_info").isNull() & (col("seller_type") == '1p') )
            elif self.config['seller_type'] == '3p':
                audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_matches.filter(col("audited_match_seller_type") == '3p'), "base_sku_uuid", "left")
                if self.config['cardinality'] == '1':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter(col("audited_match_seller_type_list").isNotNull() | (col("seller_type") == '1p')).\
                                    withColumn("misc_info", when( col("seller_type") == '1p', lit("seller_type_config_3p")).otherwise(col("misc_info")) )
                    match_suggestion = audited_base_product_unaudited_matches.filter(col("misc_info").isNull() & (col("seller_type") == '3p'))
                if self.config['cardinality'] == 'n':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter(col("seller_type") == '1p').withColumn("misc_info", lit("seller_type_config_3p"))
                    match_suggestion = audited_base_product_unaudited_matches.filter(col("seller_type") == '3p')

            elif self.config['seller_type'] == '1p_or_3p':
                exact_matches = exact_matches.withColumn("audited_match_seller_type_list", when( col("audited_match_seller_type_list") == "1p_3p", "1p").otherwise(col("audited_match_seller_type_list")) ).\
                                            filter( col("audited_match_seller_type_list") == col("audited_match_seller_type"))
                audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_matches, "base_sku_uuid", "left")
                if self.config['cardinality'] == '1':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter(col("audited_match_seller_type_list").isNotNull() )
                    match_suggestion = audited_base_product_unaudited_matches.filter(col("audited_match_seller_type_list").isNull() )
                if self.config['cardinality'] == 'n':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNotNull() & (col("seller_type") == "1p" )  & (col("audited_match_seller_type") == "1p" )) 
                    match_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNull() | (col("seller_type") != "1p" )  | (col("audited_match_seller_type") != "1p" ) ) 
            
            elif self.config['seller_type'] == '1p_over_3p':
                exact_matches = exact_matches.withColumn("audited_match_seller_type_list", when( col("audited_match_seller_type_list") == "1p_3p", "1p").otherwise(col("audited_match_seller_type_list")) ).\
                                            filter( col("audited_match_seller_type_list") == col("audited_match_seller_type"))
                audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_matches, "base_sku_uuid", "left")
                if self.config['cardinality'] == '1':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter( ~(col("audited_match_seller_type_list").isNull() | ((col("audited_match_seller_type_list") == "3p") & ( col('seller_type') == '1p' )) ) )
                    match_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNull() | ( (col("audited_match_seller_type_list") == "3p") & (col('seller_type') == '1p') )  )
                if self.config['cardinality'] == 'n':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNotNull() & (col("seller_type") == "1p" )  & (col("audited_match_seller_type") == "1p" )) 
                    match_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNull() | (col("seller_type") != "1p" )  | (col("audited_match_seller_type") != "1p" ) ) 

        elif self.config['match_type'] == 'similar' or self.config['match_type'] == 'exact_or_similar':
            exact_and_similar_match = exact_matches.union(similar_matches)
            if self.config['seller_type'] == '1p':
                audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_and_similar_match.filter(col("audited_match_seller_type") == '1p'), "base_sku_uuid", "left")
                if self.config['cardinality'] == '1':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNotNull() | (col("seller_type") == '3p')).\
                                    withColumn("misc_info", when( col("seller_type") == '3p', lit("seller_type_config_1p")).otherwise(col("misc_info")) )
                    match_suggestion = audited_base_product_unaudited_matches.filter( col("misc_info").isNull() & (col("seller_type") == '1p') )
                if self.config['cardinality'] == 'n':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter( (col("seller_type") == '3p')).\
                                    withColumn("misc_info", when( col("seller_type") == '3p', lit("seller_type_config_3p")).otherwise(col("misc_info")) )
                    match_suggestion = audited_base_product_unaudited_matches.filter( (col("seller_type") == '1p'))
            elif self.config['seller_type'] == '3p':
                audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_and_similar_match.filter(col("audited_match_seller_type") == '3p'), "base_sku_uuid", "left")
                audited_base_product_unaudited_matches.show()
                if self.config['cardinality'] == '1':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNotNull() | (col("seller_type") == '1p')).\
                                    withColumn("misc_info", when( col("seller_type") == '1p', lit("seller_type_config_1p")).otherwise(col("misc_info")) )
                    match_suggestion = audited_base_product_unaudited_matches.filter( col("misc_info").isNull() & (col("seller_type") == '3p') )
                if self.config['cardinality'] == 'n':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter( (col("seller_type") == '1p')).\
                                    withColumn("misc_info", when( col("seller_type") == '1p', lit("seller_type_config_3p")).otherwise(col("misc_info")) )
                    match_suggestion = audited_base_product_unaudited_matches.filter( (col("seller_type") == '3p'))
            elif self.config['seller_type'] == '1p_or_3p':
                exact_and_similar_match = exact_and_similar_match.withColumn("audited_match_seller_type_list", when( col("audited_match_seller_type_list") == "1p_3p", "1p").otherwise(col("audited_match_seller_type_list")) ).\
                                            filter( col("audited_match_seller_type_list") == col("audited_match_seller_type"))
                audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_and_similar_match, "base_sku_uuid", "left")
                if self.config['cardinality'] == '1':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter(col("audited_match_seller_type_list").isNotNull() )
                    match_suggestion = audited_base_product_unaudited_matches.filter(col("audited_match_seller_type_list").isNull() )
                if self.config['cardinality'] == 'n':
                    pruned_suggestion = None
                    match_suggestion = audited_base_product_unaudited_matches
            elif self.config['seller_type'] == '1p_over_3p':
                exact_and_similar_match = exact_and_similar_match.withColumn("audited_match_seller_type_list", when( col("audited_match_seller_type_list") == "1p_3p", "1p").otherwise(col("audited_match_seller_type_list")) ).\
                                            filter( col("audited_match_seller_type_list") == col("audited_match_seller_type"))
                audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_and_similar_match, "base_sku_uuid", "left")
                audited_base_product_unaudited_matches.printSchema()
                if self.config['cardinality'] == '1':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter( ~(col("audited_match_seller_type_list").isNull() | ((col("audited_match_seller_type_list") == "3p") & ( col('seller_type') == '1p' )) ) )
                    match_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNull() | ( (col("audited_match_seller_type_list") == "3p") & (col('seller_type') == '1p') )  )
                if self.config['cardinality'] == 'n':
                    pruned_suggestion = None
                    match_suggestion = audited_base_product_unaudited_matches
        return pruned_suggestion, match_suggestion
        
    

In [82]:
config = {
    "seller_type" : "1p",
    "cardinality" : "n",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
match_suggestion.show(truncate=False)
pruned_suggestion.show(truncate=False)

separate_audited_and_unaudited_matches
+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_seller_type|
+--------------------+------------------------------+--------------------+-------------------------+
|2<>100670057<>hom...|                            1p|similar_match_2<>...|                       1p|
|3<>100341666<>hom...|                            3p|similar_match_3<>...|                       3p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       1p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       3p|
+--------------------+------------------------------+--------------------+-------------------------+

+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_lis

In [83]:
config = {
    "seller_type" : "1p",
    "cardinality" : "1",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
match_suggestion.show(truncate=False)
pruned_suggestion.show(truncate=False)

separate_audited_and_unaudited_matches
+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_seller_type|
+--------------------+------------------------------+--------------------+-------------------------+
|2<>100670057<>hom...|                            1p|similar_match_2<>...|                       1p|
|3<>100341666<>hom...|                            3p|similar_match_3<>...|                       3p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       1p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       3p|
+--------------------+------------------------------+--------------------+-------------------------+

+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_lis

In [86]:
config = {
    "seller_type" : "3p",
    "cardinality" : "1",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
match_suggestion.show(truncate=False)
pruned_suggestion.show(truncate=False)

n = 1
matche_data = [
    ["0000346318<>100341666<>homedepot<>homedepot_0003463186541<>109589<>bcg<>bcg","0000346318<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","0003463186541<>109589<>bcg<>bcg", "bcg<>bcg","3P"],
    ["00003463186541<>100341666<>homedepot<>homedepot_00034631<>109589<>bcg<>bcg","00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","00034631<>109589<>bcg<>bcg","bcg<>bcg","3P"],
    ["0000000001<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy","0000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy", "chewy<>chewy","3P"],
]

separate_audited_and_unaudited_matches
+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_seller_type|
+--------------------+------------------------------+--------------------+-------------------------+
|2<>100670057<>hom...|                            1p|similar_match_2<>...|                       1p|
|3<>100341666<>hom...|                            3p|similar_match_3<>...|                       3p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       1p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       3p|
+--------------------+------------------------------+--------------------+-------------------------+

+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_lis

In [87]:
config = {
    "seller_type" : "3p",
    "cardinality" : "n",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
match_suggestion.show(truncate=False)
pruned_suggestion.show(truncate=False)


separate_audited_and_unaudited_matches
+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_seller_type|
+--------------------+------------------------------+--------------------+-------------------------+
|2<>100670057<>hom...|                            1p|similar_match_2<>...|                       1p|
|3<>100341666<>hom...|                            3p|similar_match_3<>...|                       3p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       1p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       3p|
+--------------------+------------------------------+--------------------+-------------------------+

+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_lis

In [35]:
config = {
    "seller_type" : "1p_or_3p",
    "cardinality" : "1",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
if match_suggestion is not None:
    match_suggestion.show(truncate=False)
else:
    print("match_suggestion is empty")

if pruned_suggestion is not None:
    pruned_suggestion.show(truncate=False)
else:
    print("pruned_suggestion is empty")

matche_data = [
    ["00003463186541<>100341666<>homedepot<>homedepot_0003463186541<>109589<>bcg<>bcg","00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","0003463186541<>109589<>bcg<>bcg", "bcg<>bcg","3P"],
    ["00003463186541<>100341666<>homedepot<>homedepot_00034631<>109589<>bcg<>bcg","00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","00034631<>109589<>bcg<>bcg","bcg<>bcg","3P"],
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy","0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy", "chewy<>chewy","3P"],
    ["0000000000001<>53409<>petcarerx<>petcarerx_0000<>147664<>chewy<>chewy","0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","0000<>147664<>chewy<>chewy", "chewy<>chewy","3P"],

]

separate_audited_and_unaudited_matches
+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_seller_type|
+--------------------+------------------------------+--------------------+-------------------------+
|2<>100670057<>hom...|                            1p|similar_match_2<>...|                       1p|
|3<>100341666<>hom...|                            3p|similar_match_3<>...|                       3p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       1p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       3p|
+--------------------+------------------------------+--------------------+-------------------------+

+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_lis

In [ ]:
+-------------------------------------+-----------------------------------------------------------------------+--------------------+-----------------------------------+-----------------+-----------+------------------------------+---------+-------------------------+
|base_sku_uuid                        |pair_id                                                                |base_source_store   |comp_sku_uuid                      |comp_source_store|seller_type|audited_match_seller_type_list|misc_info|audited_match_seller_type|
+-------------------------------------+-----------------------------------------------------------------------+--------------------+-----------------------------------+-----------------+-----------+------------------------------+---------+-------------------------+
|new3<>53409<>petcarerx<>petcarerx    |new3<>53409<>petcarerx<>petcarerx_compNoSugg3p1<>147664<>chewy<>chewy  |petcarerx<>petcarerx|compNoSugg3p1<>147664<>chewy<>chewy|chewy<>chewy     |3p         |null                          |null     |null                     |
|6<>100670057<>homedepot<>homedepot   |6<>100670057<>homedepot<>homedepot_compNo1pSimSugg1p<>967493<>bcg<>bcg |homedepot<>homedepot|compNo1pSimSugg1p<>967493<>bcg<>bcg|bcg<>bcg         |1p         |null                          |null     |null                     |
|6<>100670057<>homedepot<>homedepot   |6<>100670057<>homedepot<>homedepot_compNo1pSimSugg3p<>967493<>bcg<>bcg |homedepot<>homedepot|compNo1pSimSugg3p<>967493<>bcg<>bcg|bcg<>bcg         |3p         |null                          |null     |null                     |
|6<>100670057<>homedepot<>homedepot   |6<>100670057<>homedepot<>homedepot_compNo3pSimSugg1p<>967493<>bcg<>bcg |homedepot<>homedepot|compNo3pSimSugg1p<>967493<>bcg<>bcg|bcg<>bcg         |1p         |null                          |null     |null                     |
|6<>100670057<>homedepot<>homedepot   |6<>100670057<>homedepot<>homedepot_compNo3pSimSugg3p<>967493<>bcg<>bcg |homedepot<>homedepot|compNo3pSimSugg3p<>967493<>bcg<>bcg|bcg<>bcg         |3p         |null                          |null     |null                     |
|new4<>100341666<>homedepot<>homedepot|new4<>100341666<>homedepot<>homedepot_compNoSugg3p2<>109589<>bcg<>bcg  |homedepot<>homedepot|compNoSugg3p2<>109589<>bcg<>bcg    |bcg<>bcg         |3p         |null                          |null     |null                     |
|3<>100341666<>homedepot<>homedepot   |3<>100341666<>homedepot<>homedepot_compNo3pSimSugg1p<>109589<>bcg<>bcg |homedepot<>homedepot|compNo3pSimSugg1p<>109589<>bcg<>bcg|bcg<>bcg         |1p         |null                          |null     |null                     |
|3<>100341666<>homedepot<>homedepot   |3<>100341666<>homedepot<>homedepot_compNo3pSimSugg3p<>109589<>bcg<>bcg |homedepot<>homedepot|compNo3pSimSugg3p<>109589<>bcg<>bcg|bcg<>bcg         |3p         |null                          |null     |null                     |
|new1<>203359001<>homedepot<>homedepot|new1<>203359001<>homedepot<>homedepot_compNoSugg1p1<>g4103325<>bcg<>bcg|homedepot<>homedepot|compNoSugg1p1<>g4103325<>bcg<>bcg  |bcg<>bcg         |1p         |null                          |null     |null                     |
|new2<>100341666<>homedepot<>homedepot|new2<>100341666<>homedepot<>homedepot_compNoSugg1p2<>109589<>bcg<>bcg  |homedepot<>homedepot|compNoSugg1p2<>109589<>bcg<>bcg    |bcg<>bcg         |1p         |null                          |null     |null                     |
|2<>100670057<>homedepot<>homedepot   |2<>100670057<>homedepot<>homedepot_compNo1pSimSugg1p<>967493<>bcg<>bcg |homedepot<>homedepot|compNo1pSimSugg1p<>967493<>bcg<>bcg|bcg<>bcg         |1p         |null                          |null     |null                     |
|2<>100670057<>homedepot<>homedepot   |2<>100670057<>homedepot<>homedepot_compNo1pSimSugg3p<>967493<>bcg<>bcg |homedepot<>homedepot|compNo1pSimSugg3p<>967493<>bcg<>bcg|bcg<>bcg         |3p         |null                          |null     |null                     |
+-------------------------------------+-----------------------------------------------------------------------+--------------------+-----------------------------------+-----------------+-----------+------------------------------+---------+-------------------------+

+----------------------------------+----------------------------------------------------------------------------+--------------------+-----------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |pair_id                                                                     |base_source_store   |comp_sku_uuid                            |comp_source_store|seller_type|audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+----------------------------------------------------------------------------+--------------------+-----------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|1<>203370169<>homedepot<>homedepot|1<>203370169<>homedepot<>homedepot_compNo1pExactSugg3p<>967446<>bcg<>bcg    |homedepot<>homedepot|compNo1pExactSugg3p<>967446<>bcg<>bcg    |bcg<>bcg         |3p         |1p                            |exact_match_1<>203370169<>homedepot<>homedepot_compYes1pExactSugg1p<>967446<>bcg<>bcg  |1p                       |
|1<>203370169<>homedepot<>homedepot|1<>203370169<>homedepot<>homedepot_compNo1pExactSugg1p<>967446<>bcg<>bcg    |homedepot<>homedepot|compNo1pExactSugg1p<>967446<>bcg<>bcg    |bcg<>bcg         |1p         |1p                            |exact_match_1<>203370169<>homedepot<>homedepot_compYes1pExactSugg1p<>967446<>bcg<>bcg  |1p                       |
|4<>53409<>petcarerx<>petcarerx    |4<>53409<>petcarerx<>petcarerx_compNo3pExactSugg3p<>147664<>chewy<>chewy    |petcarerx<>petcarerx|compNo3pExactSugg3p<>147664<>chewy<>chewy|chewy<>chewy     |3p         |3p                            |exact_match_4<>53409<>petcarerx<>petcarerx_compYes3pExactSugg3p<>147664<>chewy<>chewy  |3p                       |
|4<>53409<>petcarerx<>petcarerx    |4<>53409<>petcarerx<>petcarerx_compNo3pExactSugg1p<>147664<>chewy<>chewy    |petcarerx<>petcarerx|compNo3pExactSugg1p<>147664<>chewy<>chewy|chewy<>chewy     |1p         |3p                            |exact_match_4<>53409<>petcarerx<>petcarerx_compYes3pExactSugg3p<>147664<>chewy<>chewy  |3p                       |
|5<>203370169<>homedepot<>homedepot|5<>203370169<>homedepot<>homedepot_compNo3pExactSugg3p<>147664<>chewy<>chewy|homedepot<>homedepot|compNo3pExactSugg3p<>147664<>chewy<>chewy|chewy<>chewy     |3p         |1p_3p                         |exact_match_5<>203370169<>homedepot<>homedepot_compYes1p3pExactSugg1p<>967446<>bcg<>bcg|1p                       |
|5<>203370169<>homedepot<>homedepot|5<>203370169<>homedepot<>homedepot_compNo3pExactSugg1p<>147664<>chewy<>chewy|homedepot<>homedepot|compNo3pExactSugg1p<>147664<>chewy<>chewy|chewy<>chewy     |1p         |1p_3p                         |exact_match_5<>203370169<>homedepot<>homedepot_compYes1p3pExactSugg1p<>967446<>bcg<>bcg|1p                       |
|5<>203370169<>homedepot<>homedepot|5<>203370169<>homedepot<>homedepot_compNo1pExactSugg3p<>967446<>bcg<>bcg    |homedepot<>homedepot|compNo1pExactSugg3p<>967446<>bcg<>bcg    |bcg<>bcg         |3p         |1p_3p                         |exact_match_5<>203370169<>homedepot<>homedepot_compYes1p3pExactSugg1p<>967446<>bcg<>bcg|1p                       |
|5<>203370169<>homedepot<>homedepot|5<>203370169<>homedepot<>homedepot_compNo1pExactSugg1p<>967446<>bcg<>bcg    |homedepot<>homedepot|compNo1pExactSugg1p<>967446<>bcg<>bcg    |bcg<>bcg         |1p         |1p_3p                         |exact_match_5<>203370169<>homedepot<>homedepot_compYes1p3pExactSugg1p<>967446<>bcg<>bcg|1p                       |
+----------------------------------+----------------------------------------------------------------------------+--------------------+-----------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+


In [58]:
config = {
    "seller_type" : "1p_or_3p",
    "cardinality" : "n",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
if match_suggestion is not None:
    match_suggestion.show(truncate=False)
else:
    print("match_suggestion is empty")

if pruned_suggestion is not None:
    pruned_suggestion.show(truncate=False)
else:
    print("pruned_suggestion is empty")

matche_data = [
    ["new1<>203359001<>homedepot<>homedepot_compNoSugg1p1<>g4103325<>bcg<>bcg","new1<>203359001<>homedepot<>homedepot","homedepot<>homedepot","compNoSugg1p1<>g4103325<>bcg<>bcg", "bcg<>bcg","1P"],
    ["new2<>100341666<>homedepot<>homedepot_compNoSugg1p2<>109589<>bcg<>bcg","new2<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compNoSugg1p2<>109589<>bcg<>bcg", "bcg<>bcg","1P"],
    
     ["new3<>53409<>petcarerx<>petcarerx_compNoSugg3p1<>147664<>chewy<>chewy","new3<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","compNoSugg3p1<>147664<>chewy<>chewy", "chewy<>chewy","3P"],
    ["new4<>100341666<>homedepot<>homedepot_compNoSugg3p2<>109589<>bcg<>bcg","new4<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compNoSugg3p2<>109589<>bcg<>bcg", "bcg<>bcg","3P"],

    ["2<>100670057<>homedepot<>homedepot_compNo1pSimSugg1p<>967493<>bcg<>bcg","2<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pSimSugg1p<>967493<>bcg<>bcg","bcg<>bcg","1P"],
    ["3<>100341666<>homedepot<>homedepot_compNo3pSimSugg3p<>109589<>bcg<>bcg","3<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compNo3pSimSugg3p<>109589<>bcg<>bcg","bcg<>bcg","3P"],
    ["4<>53409<>petcarerx<>petcarerx_compNo3pExactSugg3p<>147664<>chewy<>chewy","4<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","compNo3pExactSugg3p<>147664<>chewy<>chewy", "chewy<>chewy","3P"],

    [ "1<>203370169<>homedepot<>homedepot_compNo1pExactSugg3p<>967446<>bcg<>bcg","1<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pExactSugg3p<>967446<>bcg<>bcg","bcg<>bcg","3P"],
    [ "2<>100670057<>homedepot<>homedepot_compNo1pSimSugg3p<>967493<>bcg<>bcg","2<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pExactSugg3p<>967493<>bcg<>bcg","bcg<>bcg","3P"],
    [ "3<>100341666<>homedepot<>homedepot_compNo3pSimSugg1p<>109589<>bcg<>bcg","3<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compNo3pExactSugg1p<>109589<>bcg<>bcg","bcg<>bcg","1P"],
    [ "4<>53409<>petcarerx<>petcarerx_compNo3pExactSugg1p<>147664<>chewy<>chewy","4<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","compNo3pExactSugg1p<>147664<>chewy<>chewy", "chewy<>chewy","1P"],
    

]

separate_audited_and_unaudited_matches
+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_seller_type|
+--------------------+------------------------------+--------------------+-------------------------+
|2<>100670057<>hom...|                            1p|similar_match_2<>...|                       1p|
|3<>100341666<>hom...|                            3p|similar_match_3<>...|                       3p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       1p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       3p|
+--------------------+------------------------------+--------------------+-------------------------+

+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_lis

In [ ]:

+-------------------------------------+----------------------------------------------------------------------------+--------------------+-----------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                        |pair_id                                                                     |base_source_store   |comp_sku_uuid                            |comp_source_store|seller_type|audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+-------------------------------------+----------------------------------------------------------------------------+--------------------+-----------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|new3<>53409<>petcarerx<>petcarerx    |new3<>53409<>petcarerx<>petcarerx_compNoSugg3p1<>147664<>chewy<>chewy       |petcarerx<>petcarerx|compNoSugg3p1<>147664<>chewy<>chewy      |chewy<>chewy     |3p         |null                          |null                                                                                   |null                     |
|4<>53409<>petcarerx<>petcarerx       |4<>53409<>petcarerx<>petcarerx_compNo3pExactSugg3p<>147664<>chewy<>chewy    |petcarerx<>petcarerx|compNo3pExactSugg3p<>147664<>chewy<>chewy|chewy<>chewy     |3p         |3p                            |exact_match_4<>53409<>petcarerx<>petcarerx_compYes3pExactSugg3p<>147664<>chewy<>chewy  |3p                       |
|6<>100670057<>homedepot<>homedepot   |6<>100670057<>homedepot<>homedepot_compNo1pSimSugg1p<>967493<>bcg<>bcg      |homedepot<>homedepot|compNo1pSimSugg1p<>967493<>bcg<>bcg      |bcg<>bcg         |1p         |null                          |null                                                                                   |null                     |
|6<>100670057<>homedepot<>homedepot   |6<>100670057<>homedepot<>homedepot_compNo1pSimSugg3p<>967493<>bcg<>bcg      |homedepot<>homedepot|compNo1pSimSugg3p<>967493<>bcg<>bcg      |bcg<>bcg         |3p         |null                          |null                                                                                   |null                     |
|6<>100670057<>homedepot<>homedepot   |6<>100670057<>homedepot<>homedepot_compNo3pSimSugg1p<>967493<>bcg<>bcg      |homedepot<>homedepot|compNo3pSimSugg1p<>967493<>bcg<>bcg      |bcg<>bcg         |1p         |null                          |null                                                                                   |null                     |
|6<>100670057<>homedepot<>homedepot   |6<>100670057<>homedepot<>homedepot_compNo3pSimSugg3p<>967493<>bcg<>bcg      |homedepot<>homedepot|compNo3pSimSugg3p<>967493<>bcg<>bcg      |bcg<>bcg         |3p         |null                          |null                                                                                   |null                     |
|new4<>100341666<>homedepot<>homedepot|new4<>100341666<>homedepot<>homedepot_compNoSugg3p2<>109589<>bcg<>bcg       |homedepot<>homedepot|compNoSugg3p2<>109589<>bcg<>bcg          |bcg<>bcg         |3p         |null                          |null                                                                                   |null                     |
|3<>100341666<>homedepot<>homedepot   |3<>100341666<>homedepot<>homedepot_compNo3pSimSugg1p<>109589<>bcg<>bcg      |homedepot<>homedepot|compNo3pSimSugg1p<>109589<>bcg<>bcg      |bcg<>bcg         |1p         |null                          |null                                                                                   |null                     |
|3<>100341666<>homedepot<>homedepot   |3<>100341666<>homedepot<>homedepot_compNo3pSimSugg3p<>109589<>bcg<>bcg      |homedepot<>homedepot|compNo3pSimSugg3p<>109589<>bcg<>bcg      |bcg<>bcg         |3p         |null                          |null                                                                                   |null                     |
|new1<>203359001<>homedepot<>homedepot|new1<>203359001<>homedepot<>homedepot_compNoSugg1p1<>g4103325<>bcg<>bcg     |homedepot<>homedepot|compNoSugg1p1<>g4103325<>bcg<>bcg        |bcg<>bcg         |1p         |null                          |null                                                                                   |null                     |
|new2<>100341666<>homedepot<>homedepot|new2<>100341666<>homedepot<>homedepot_compNoSugg1p2<>109589<>bcg<>bcg       |homedepot<>homedepot|compNoSugg1p2<>109589<>bcg<>bcg          |bcg<>bcg         |1p         |null                          |null                                                                                   |null                     |
|1<>203370169<>homedepot<>homedepot   |1<>203370169<>homedepot<>homedepot_compNo1pExactSugg3p<>967446<>bcg<>bcg    |homedepot<>homedepot|compNo1pExactSugg3p<>967446<>bcg<>bcg    |bcg<>bcg         |3p         |1p                            |exact_match_1<>203370169<>homedepot<>homedepot_compYes1pExactSugg1p<>967446<>bcg<>bcg  |1p                       |
|5<>203370169<>homedepot<>homedepot   |5<>203370169<>homedepot<>homedepot_compNo1pExactSugg3p<>967446<>bcg<>bcg    |homedepot<>homedepot|compNo1pExactSugg3p<>967446<>bcg<>bcg    |bcg<>bcg         |3p         |1p                            |exact_match_5<>203370169<>homedepot<>homedepot_compYes1p3pExactSugg1p<>967446<>bcg<>bcg|1p                       |
|5<>203370169<>homedepot<>homedepot   |5<>203370169<>homedepot<>homedepot_compNo3pExactSugg3p<>147664<>chewy<>chewy|homedepot<>homedepot|compNo3pExactSugg3p<>147664<>chewy<>chewy|chewy<>chewy     |3p         |1p                            |exact_match_5<>203370169<>homedepot<>homedepot_compYes1p3pExactSugg1p<>967446<>bcg<>bcg|1p                       |
|2<>100670057<>homedepot<>homedepot   |2<>100670057<>homedepot<>homedepot_compNo1pSimSugg1p<>967493<>bcg<>bcg      |homedepot<>homedepot|compNo1pSimSugg1p<>967493<>bcg<>bcg      |bcg<>bcg         |1p         |null                          |null                                                                                   |null                     |
|2<>100670057<>homedepot<>homedepot   |2<>100670057<>homedepot<>homedepot_compNo1pSimSugg3p<>967493<>bcg<>bcg      |homedepot<>homedepot|compNo1pSimSugg3p<>967493<>bcg<>bcg      |bcg<>bcg         |3p         |null                          |null                                                                                   |null                     |
+-------------------------------------+----------------------------------------------------------------------------+--------------------+-----------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+

+----------------------------------+----------------------------------------------------------------------------+--------------------+-----------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |pair_id                                                                     |base_source_store   |comp_sku_uuid                            |comp_source_store|seller_type|audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+----------------------------------------------------------------------------+--------------------+-----------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|1<>203370169<>homedepot<>homedepot|1<>203370169<>homedepot<>homedepot_compNo1pExactSugg1p<>967446<>bcg<>bcg    |homedepot<>homedepot|compNo1pExactSugg1p<>967446<>bcg<>bcg    |bcg<>bcg         |1p         |1p                            |exact_match_1<>203370169<>homedepot<>homedepot_compYes1pExactSugg1p<>967446<>bcg<>bcg  |1p                       |
|4<>53409<>petcarerx<>petcarerx    |4<>53409<>petcarerx<>petcarerx_compNo3pExactSugg1p<>147664<>chewy<>chewy    |petcarerx<>petcarerx|compNo3pExactSugg1p<>147664<>chewy<>chewy|chewy<>chewy     |1p         |3p                            |exact_match_4<>53409<>petcarerx<>petcarerx_compYes3pExactSugg3p<>147664<>chewy<>chewy  |3p                       |
|5<>203370169<>homedepot<>homedepot|5<>203370169<>homedepot<>homedepot_compNo3pExactSugg1p<>147664<>chewy<>chewy|homedepot<>homedepot|compNo3pExactSugg1p<>147664<>chewy<>chewy|chewy<>chewy     |1p         |1p                            |exact_match_5<>203370169<>homedepot<>homedepot_compYes1p3pExactSugg1p<>967446<>bcg<>bcg|1p                       |
|5<>203370169<>homedepot<>homedepot|5<>203370169<>homedepot<>homedepot_compNo1pExactSugg1p<>967446<>bcg<>bcg    |homedepot<>homedepot|compNo1pExactSugg1p<>967446<>bcg<>bcg    |bcg<>bcg         |1p         |1p                            |exact_match_5<>203370169<>homedepot<>homedepot_compYes1p3pExactSugg1p<>967446<>bcg<>bcg|1p                       |
+----------------------------------+----------------------------------------------------------------------------+--------------------+-----------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+


In [61]:
config = {
    "seller_type" : "1p_over_3p",
    "cardinality" : "1",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
if match_suggestion is not None:
    match_suggestion.show(truncate=False)
else:
    print("match_suggestion is empty")

if pruned_suggestion is not None:
    pruned_suggestion.show(truncate=False)
else:
    print("pruned_suggestion is empty")

separate_audited_and_unaudited_matches
+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_seller_type|
+--------------------+------------------------------+--------------------+-------------------------+
|2<>100670057<>hom...|                            1p|similar_match_2<>...|                       1p|
|3<>100341666<>hom...|                            3p|similar_match_3<>...|                       3p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       1p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       3p|
+--------------------+------------------------------+--------------------+-------------------------+

+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_lis

In [64]:
config = {
    "seller_type" : "1p_over_3p",
    "cardinality" : "n",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
if match_suggestion is not None:
    match_suggestion.show(truncate=False)
else:
    print("match_suggestion is empty")

if pruned_suggestion is not None:
    pruned_suggestion.show(truncate=False)
else:
    print("pruned_suggestion is empty")

separate_audited_and_unaudited_matches
+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_seller_type|
+--------------------+------------------------------+--------------------+-------------------------+
|2<>100670057<>hom...|                            1p|similar_match_2<>...|                       1p|
|3<>100341666<>hom...|                            3p|similar_match_3<>...|                       3p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       1p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       3p|
+--------------------+------------------------------+--------------------+-------------------------+

+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_lis

In [19]:
str_1 = "1p"
str_2 = "1p"
str_3 = "3p"

print(str_1 == str_2)
print(str_1 > str_3)
print(str_2 > str_3)

True
False
False


In [15]:
config = {
    "seller_type" : "1p",
    "cardinality" : "1",
    "match_type" : "similar"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
if match_suggestion is not None:
    match_suggestion.show(truncate=False)
else:
    print("match_suggestion is empty")

if pruned_suggestion is not None:
    pruned_suggestion.show(truncate=False)
else:
    print("pruned_suggestion is empty")

separate_audited_and_unaudited_matches
+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_seller_type|
+--------------------+------------------------------+--------------------+-------------------------+
|2<>100670057<>hom...|                            1p|similar_match_2<>...|                       1p|
|3<>100341666<>hom...|                            3p|similar_match_3<>...|                       3p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       1p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       3p|
+--------------------+------------------------------+--------------------+-------------------------+

+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_lis

In [14]:
config = {
    "seller_type" : "1p",
    "cardinality" : "n",
    "match_type" : "similar"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
if match_suggestion is not None:
    match_suggestion.show(truncate=False)
else:
    print("match_suggestion is empty")

if pruned_suggestion is not None:
    pruned_suggestion.show(truncate=False)
else:
    print("pruned_suggestion is empty")

separate_audited_and_unaudited_matches
+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_seller_type|
+--------------------+------------------------------+--------------------+-------------------------+
|2<>100670057<>hom...|                            1p|similar_match_2<>...|                       1p|
|3<>100341666<>hom...|                            3p|similar_match_3<>...|                       3p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       1p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       3p|
+--------------------+------------------------------+--------------------+-------------------------+

+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_lis

In [22]:
config = {
    "seller_type" : "3p",
    "cardinality" : "1",
    "match_type" : "similar"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
if match_suggestion is not None:
    match_suggestion.show(truncate=False)
else:
    print("match_suggestion is empty")

if pruned_suggestion is not None:
    pruned_suggestion.show(truncate=False)
else:
    print("pruned_suggestion is empty")

separate_audited_and_unaudited_matches


+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_seller_type|
+--------------------+------------------------------+--------------------+-------------------------+
|2<>100670057<>hom...|                            1p|similar_match_2<>...|                       1p|
|3<>100341666<>hom...|                            3p|similar_match_3<>...|                       3p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       1p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       3p|
+--------------------+------------------------------+--------------------+-------------------------+

+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_se

In [31]:
config = {
    "seller_type" : "3p",
    "cardinality" : "n",
    "match_type" : "similar"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
if match_suggestion is not None:
    match_suggestion.show(truncate=False)
else:
    print("match_suggestion is empty")

if pruned_suggestion is not None:
    pruned_suggestion.show(truncate=False)
else:
    print("pruned_suggestion is empty")

separate_audited_and_unaudited_matches
+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_seller_type|
+--------------------+------------------------------+--------------------+-------------------------+
|2<>100670057<>hom...|                            1p|similar_match_2<>...|                       1p|
|3<>100341666<>hom...|                            3p|similar_match_3<>...|                       3p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       1p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       3p|
+--------------------+------------------------------+--------------------+-------------------------+

+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_lis

In [32]:
config = {
    "seller_type" : "1p_or_3p",
    "cardinality" : "1",
    "match_type" : "similar"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
if match_suggestion is not None:
    match_suggestion.show(truncate=False)
else:
    print("match_suggestion is empty")

if pruned_suggestion is not None:
    pruned_suggestion.show(truncate=False)
else:
    print("pruned_suggestion is empty")

separate_audited_and_unaudited_matches
+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_seller_type|
+--------------------+------------------------------+--------------------+-------------------------+
|2<>100670057<>hom...|                            1p|similar_match_2<>...|                       1p|
|3<>100341666<>hom...|                            3p|similar_match_3<>...|                       3p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       1p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       3p|
+--------------------+------------------------------+--------------------+-------------------------+

+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_lis

In [33]:
config = {
    "seller_type" : "1p_or_3p",
    "cardinality" : "n",
    "match_type" : "similar"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
if match_suggestion is not None:
    match_suggestion.show(truncate=False)
else:
    print("match_suggestion is empty")

if pruned_suggestion is not None:
    pruned_suggestion.show(truncate=False)
else:
    print("pruned_suggestion is empty")

separate_audited_and_unaudited_matches
+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_seller_type|
+--------------------+------------------------------+--------------------+-------------------------+
|2<>100670057<>hom...|                            1p|similar_match_2<>...|                       1p|
|3<>100341666<>hom...|                            3p|similar_match_3<>...|                       3p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       1p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       3p|
+--------------------+------------------------------+--------------------+-------------------------+

+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_lis

In [37]:
config = {
    "seller_type" : "1p_over_3p",
    "cardinality" : "1",
    "match_type" : "similar"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
if match_suggestion is not None:
    match_suggestion.show(truncate=False)
else:
    print("match_suggestion is empty")

if pruned_suggestion is not None:
    pruned_suggestion.show(truncate=False)
else:
    print("pruned_suggestion is empty")

separate_audited_and_unaudited_matches
+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_seller_type|
+--------------------+------------------------------+--------------------+-------------------------+
|2<>100670057<>hom...|                            1p|similar_match_2<>...|                       1p|
|3<>100341666<>hom...|                            3p|similar_match_3<>...|                       3p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       1p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       3p|
+--------------------+------------------------------+--------------------+-------------------------+

+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_lis

In [38]:
config = {
    "seller_type" : "1p_over_3p",
    "cardinality" : "n",
    "match_type" : "similar"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
if match_suggestion is not None:
    match_suggestion.show(truncate=False)
else:
    print("match_suggestion is empty")

if pruned_suggestion is not None:
    pruned_suggestion.show(truncate=False)
else:
    print("pruned_suggestion is empty")

separate_audited_and_unaudited_matches
+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_list|           misc_info|audited_match_seller_type|
+--------------------+------------------------------+--------------------+-------------------------+
|2<>100670057<>hom...|                            1p|similar_match_2<>...|                       1p|
|3<>100341666<>hom...|                            3p|similar_match_3<>...|                       3p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       1p|
|6<>100670057<>hom...|                         1p_3p|similar_match_6<>...|                       3p|
+--------------------+------------------------------+--------------------+-------------------------+

+--------------------+------------------------------+--------------------+-------------------------+
|       base_sku_uuid|audited_match_seller_type_lis